<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [6]:
%%writefile Lotus.py
from selenium import webdriver
from selenium.webdriver.common.by import By
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/grocery"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.get_href)

    def get_href(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(
                    By.XPATH, "//div[@class='carousel']")
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)

        for p in selector.xpath(
                "//a[contains(@class, 'MuiButtonBase-root')]"):
            page = p.xpath("./@href").get()
            if page is not None:
                yield response.follow(page, callback=self.get_href)

    def parse_item(self, response):
        print(response.url)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [7]:
%%bash
python Lotus.py

Process is terminated.
